## Hồi quy tuyến tính và hồi quy đa thức để định giá bí ngô - Bài học 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Đồ họa thông tin bởi Dasani Madipalli</figcaption>


<!--![Đồ họa thông tin bởi Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Giới thiệu

Cho đến nay, bạn đã tìm hiểu về hồi quy với dữ liệu mẫu được thu thập từ tập dữ liệu định giá bí ngô mà chúng ta sẽ sử dụng xuyên suốt bài học này. Bạn cũng đã trực quan hóa nó bằng cách sử dụng `ggplot2`.💪

Bây giờ bạn đã sẵn sàng đi sâu hơn vào hồi quy trong học máy. Trong bài học này, bạn sẽ tìm hiểu thêm về hai loại hồi quy: *hồi quy tuyến tính cơ bản* và *hồi quy đa thức*, cùng với một số toán học cơ bản liên quan đến các kỹ thuật này.

> Xuyên suốt chương trình học này, chúng tôi giả định rằng bạn có kiến thức toán học tối thiểu và cố gắng làm cho nó dễ tiếp cận hơn đối với học viên đến từ các lĩnh vực khác, vì vậy hãy chú ý đến các ghi chú, 🧮 các điểm nhấn, sơ đồ và các công cụ học tập khác để hỗ trợ việc hiểu bài.

#### Chuẩn bị

Như đã nhắc lại, bạn đang tải dữ liệu này để đặt câu hỏi về nó.

-   Khi nào là thời điểm tốt nhất để mua bí ngô?

-   Giá của một thùng bí ngô nhỏ sẽ là bao nhiêu?

-   Tôi nên mua chúng trong giỏ nửa bushel hay trong hộp 1 1/9 bushel? Hãy tiếp tục khám phá dữ liệu này.

Trong bài học trước, bạn đã tạo một `tibble` (một cách tái hiện hiện đại của khung dữ liệu) và điền vào đó một phần của tập dữ liệu gốc, chuẩn hóa giá theo bushel. Tuy nhiên, bằng cách làm như vậy, bạn chỉ có thể thu thập khoảng 400 điểm dữ liệu và chỉ trong các tháng mùa thu. Có lẽ chúng ta có thể tìm hiểu thêm chi tiết về bản chất của dữ liệu bằng cách làm sạch nó hơn nữa? Chúng ta sẽ xem... 🕵️‍♀️

Để thực hiện nhiệm vụ này, chúng ta sẽ cần các gói sau:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) là một [bộ sưu tập các gói R](https://www.tidyverse.org/packages) được thiết kế để làm cho khoa học dữ liệu nhanh hơn, dễ dàng hơn và thú vị hơn!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) là một [khung làm việc](https://www.tidymodels.org/packages/) bao gồm các gói dành cho mô hình hóa và học máy.

-   `janitor`: [gói janitor](https://github.com/sfirke/janitor) cung cấp các công cụ đơn giản để kiểm tra và làm sạch dữ liệu bẩn.

-   `corrplot`: [gói corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) cung cấp một công cụ trực quan để khám phá ma trận tương quan, hỗ trợ sắp xếp lại biến tự động nhằm giúp phát hiện các mẫu ẩn giữa các biến.

Bạn có thể cài đặt chúng bằng lệnh:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Đoạn mã dưới đây kiểm tra xem bạn đã có các gói cần thiết để hoàn thành module này chưa và cài đặt chúng cho bạn nếu chúng bị thiếu.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Chúng ta sẽ tải các gói tuyệt vời này và làm cho chúng khả dụng trong phiên làm việc R hiện tại của chúng ta. (Đây chỉ là minh họa, `pacman::p_load()` đã làm điều này cho bạn)

## 1. Đường hồi quy tuyến tính

Như bạn đã học trong Bài học 1, mục tiêu của bài tập hồi quy tuyến tính là vẽ được một *đường* *phù hợp nhất* để:

-   **Hiển thị mối quan hệ giữa các biến**. Hiển thị mối quan hệ giữa các biến.

-   **Dự đoán**. Dự đoán chính xác vị trí mà một điểm dữ liệu mới sẽ nằm trong mối quan hệ với đường đó.

Để vẽ loại đường này, chúng ta sử dụng một kỹ thuật thống kê gọi là **Hồi quy Bình phương Tối thiểu**. Thuật ngữ `bình phương tối thiểu` có nghĩa là tất cả các điểm dữ liệu xung quanh đường hồi quy được bình phương và sau đó cộng lại. Lý tưởng nhất, tổng cuối cùng này càng nhỏ càng tốt, vì chúng ta muốn số lỗi thấp, hay `bình phương tối thiểu`. Do đó, đường phù hợp nhất là đường cho chúng ta giá trị thấp nhất của tổng các lỗi bình phương - vì vậy mới có tên gọi *hồi quy bình phương tối thiểu*.

Chúng ta làm điều này vì muốn mô hình hóa một đường có khoảng cách tích lũy nhỏ nhất từ tất cả các điểm dữ liệu của chúng ta. Chúng ta cũng bình phương các giá trị trước khi cộng chúng lại vì chúng ta quan tâm đến độ lớn của chúng hơn là hướng của chúng.

> **🧮 Hiển thị toán học**
>
> Đường này, gọi là *đường phù hợp nhất*, có thể được biểu diễn bằng [một phương trình](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` là '`biến giải thích` hoặc `biến dự đoán`'. `Y` là '`biến phụ thuộc` hoặc `kết quả`'. Độ dốc của đường là `b` và `a` là giao điểm trục y, tức là giá trị của `Y` khi `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "slope = $y/x$")
    Đồ họa thông tin bởi Jen Looper
>
> Đầu tiên, tính độ dốc `b`.
>
> Nói cách khác, và liên hệ với câu hỏi ban đầu về dữ liệu bí ngô của chúng ta: "dự đoán giá của một giạ bí ngô theo tháng", `X` sẽ là giá và `Y` sẽ là tháng bán.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Đồ họa thông tin bởi Jen Looper
> 
> Tính giá trị của Y. Nếu bạn đang trả khoảng \$4, chắc hẳn là tháng Tư!
>
> Phép toán tính toán đường này phải thể hiện độ dốc của đường, điều này cũng phụ thuộc vào giao điểm, hoặc vị trí của `Y` khi `X = 0`.
>
> Bạn có thể quan sát phương pháp tính toán các giá trị này trên trang web [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Cũng hãy ghé thăm [máy tính hồi quy bình phương tối thiểu này](https://www.mathsisfun.com/data/least-squares-calculator.html) để xem cách các giá trị số ảnh hưởng đến đường.

Không quá đáng sợ, đúng không? 🤓

#### Tương quan

Một thuật ngữ nữa cần hiểu là **Hệ số Tương quan** giữa các biến X và Y cho trước. Sử dụng biểu đồ phân tán, bạn có thể nhanh chóng hình dung hệ số này. Một biểu đồ với các điểm dữ liệu phân tán theo một đường gọn gàng có tương quan cao, nhưng một biểu đồ với các điểm dữ liệu phân tán khắp nơi giữa X và Y có tương quan thấp.

Một mô hình hồi quy tuyến tính tốt sẽ là mô hình có Hệ số Tương quan cao (gần 1 hơn 0) sử dụng phương pháp Hồi quy Bình phương Tối thiểu với một đường hồi quy.


## **2. Một điệu nhảy với dữ liệu: tạo một khung dữ liệu sẽ được sử dụng để mô hình hóa**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Tác phẩm nghệ thuật của @allison_horst</figcaption>


<!--![Tác phẩm nghệ thuật của \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Tải các thư viện cần thiết và tập dữ liệu. Chuyển dữ liệu thành một khung dữ liệu chứa một phần của tập dữ liệu:

-   Chỉ lấy bí ngô được định giá theo đơn vị bushel

-   Chuyển đổi ngày thành tháng

-   Tính giá trung bình dựa trên giá cao và giá thấp

-   Chuyển đổi giá để phản ánh mức giá theo số lượng bushel

> Chúng ta đã đề cập đến các bước này trong [bài học trước](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

Với tinh thần phiêu lưu thuần túy, hãy cùng khám phá [`gói janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) cung cấp các hàm đơn giản để kiểm tra và làm sạch dữ liệu bẩn. Ví dụ, hãy xem qua tên các cột trong dữ liệu của chúng ta:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Chúng ta có thể làm tốt hơn. Hãy làm cho các tên cột này `friendR` bằng cách chuyển chúng sang quy ước [snake_case](https://en.wikipedia.org/wiki/Snake_case) bằng cách sử dụng `janitor::clean_names`. Để tìm hiểu thêm về hàm này: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Rất gọn gàng 🧹! Bây giờ, hãy nhảy múa với dữ liệu bằng cách sử dụng `dplyr` như trong bài học trước! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Làm tốt lắm!👌 Giờ bạn đã có một tập dữ liệu sạch sẽ, gọn gàng để xây dựng mô hình hồi quy mới của mình!

Bạn có muốn xem biểu đồ phân tán không?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Một biểu đồ phân tán nhắc nhở chúng ta rằng chúng ta chỉ có dữ liệu tháng từ tháng Tám đến tháng Mười Hai. Chúng ta có lẽ cần thêm dữ liệu để có thể đưa ra kết luận theo cách tuyến tính.

Hãy cùng xem lại dữ liệu mô hình của chúng ta:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Nếu chúng ta muốn dự đoán `price` của một quả bí ngô dựa trên các cột `city` hoặc `package`, vốn thuộc loại ký tự, thì sao? Hoặc đơn giản hơn, làm thế nào để tìm mối tương quan (yêu cầu cả hai đầu vào phải là số) giữa, ví dụ, `package` và `price`? 🤷🤷

Các mô hình học máy hoạt động tốt nhất với các đặc trưng dạng số thay vì giá trị văn bản, vì vậy bạn thường cần chuyển đổi các đặc trưng phân loại thành các biểu diễn dạng số.

Điều này có nghĩa là chúng ta phải tìm cách định dạng lại các biến dự đoán của mình để làm cho chúng dễ sử dụng hơn đối với mô hình, một quá trình được gọi là `feature engineering`.


## 3. Tiền xử lý dữ liệu để mô hình hóa với recipes 👩‍🍳👨‍🍳

Các hoạt động định dạng lại giá trị dự đoán để giúp mô hình sử dụng hiệu quả hơn được gọi là `kỹ thuật đặc trưng`.

Các mô hình khác nhau có yêu cầu tiền xử lý khác nhau. Ví dụ, phương pháp bình phương tối thiểu yêu cầu `mã hóa các biến phân loại` như tháng, loại và tên thành phố. Điều này đơn giản chỉ là `chuyển đổi` một cột với các `giá trị phân loại` thành một hoặc nhiều `cột số` thay thế cho cột ban đầu.

Ví dụ, giả sử dữ liệu của bạn bao gồm đặc trưng phân loại sau:

|  thành phố   |
|:------------:|
|   Denver     |
|   Nairobi    |
|    Tokyo     |

Bạn có thể áp dụng *mã hóa thứ tự* để thay thế một giá trị số nguyên duy nhất cho mỗi danh mục, như sau:

| thành phố |
|:---------:|
|     0     |
|     1     |
|     2     |

Và đó chính là điều chúng ta sẽ làm với dữ liệu của mình!

Trong phần này, chúng ta sẽ khám phá một gói tuyệt vời khác của Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - được thiết kế để giúp bạn tiền xử lý dữ liệu **trước khi** huấn luyện mô hình. Cốt lõi của nó, một recipe là một đối tượng định nghĩa các bước cần áp dụng cho một tập dữ liệu để chuẩn bị cho việc mô hình hóa.

Bây giờ, hãy tạo một recipe để chuẩn bị dữ liệu của chúng ta cho việc mô hình hóa bằng cách thay thế một số nguyên duy nhất cho tất cả các quan sát trong các cột dự đoán:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Tuyệt vời! 👏 Chúng ta vừa tạo ra công thức đầu tiên để xác định một kết quả (giá cả) và các yếu tố dự đoán tương ứng, đồng thời tất cả các cột dự đoán đều được mã hóa thành một tập hợp các số nguyên 🙌! Hãy cùng phân tích nhanh:

-   Lệnh gọi `recipe()` với một công thức cho phép công thức xác định *vai trò* của các biến bằng cách sử dụng dữ liệu `new_pumpkins` làm tham chiếu. Ví dụ, cột `price` được gán vai trò là `outcome`, trong khi các cột còn lại được gán vai trò là `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` chỉ định rằng tất cả các yếu tố dự đoán sẽ được chuyển đổi thành một tập hợp các số nguyên, bắt đầu đánh số từ 0.

Chúng tôi chắc chắn rằng bạn có thể đang nghĩ: "Thật tuyệt vời!! Nhưng nếu tôi cần xác nhận rằng các công thức đang thực hiện đúng như mong đợi thì sao? 🤔"

Đó là một ý tưởng tuyệt vời! Bạn thấy đấy, một khi công thức của bạn được định nghĩa, bạn có thể ước tính các tham số cần thiết để thực sự tiền xử lý dữ liệu, sau đó trích xuất dữ liệu đã được xử lý. Bạn thường không cần làm điều này khi sử dụng Tidymodels (chúng ta sẽ thấy cách thông thường ngay sau đây-\> `workflows`), nhưng nó có thể rất hữu ích khi bạn muốn kiểm tra nhanh để xác nhận rằng các công thức đang hoạt động đúng như mong đợi.

Để làm điều đó, bạn sẽ cần hai động từ nữa: `prep()` và `bake()`. Và như mọi khi, những người bạn nhỏ R của chúng ta từ [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) sẽ giúp bạn hiểu rõ hơn về điều này!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Tác phẩm nghệ thuật của @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): ước tính các tham số cần thiết từ tập dữ liệu huấn luyện, sau đó có thể áp dụng cho các tập dữ liệu khác. Ví dụ, đối với một cột dự đoán cụ thể, quan sát nào sẽ được gán giá trị nguyên 0, 1, 2, v.v.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): sử dụng công thức đã được chuẩn bị và áp dụng các thao tác cho bất kỳ tập dữ liệu nào.

Vậy nên, hãy chuẩn bị và áp dụng công thức của chúng ta để thực sự xác nhận rằng, trong quá trình xử lý, các cột dự đoán sẽ được mã hóa trước khi mô hình được huấn luyện.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 Dữ liệu đã xử lý `baked_pumpkins` đã được mã hóa tất cả các biến dự đoán, xác nhận rằng các bước tiền xử lý được định nghĩa trong công thức của chúng ta hoạt động đúng như mong đợi. Điều này khiến việc đọc dữ liệu trở nên khó khăn hơn đối với bạn nhưng lại dễ hiểu hơn nhiều đối với Tidymodels! Hãy dành chút thời gian để tìm hiểu xem quan sát nào đã được ánh xạ thành số nguyên tương ứng.

Cũng cần lưu ý rằng `baked_pumpkins` là một khung dữ liệu mà chúng ta có thể thực hiện các phép tính trên đó.

Ví dụ, hãy thử tìm mối tương quan tốt giữa hai điểm dữ liệu của bạn để có thể xây dựng một mô hình dự đoán tốt. Chúng ta sẽ sử dụng hàm `cor()` để làm điều này. Gõ `?cor()` để tìm hiểu thêm về hàm này.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Hóa ra, mối tương quan giữa Thành phố và Giá chỉ ở mức yếu. Tuy nhiên, có một mối tương quan tốt hơn giữa Gói hàng và Giá của nó. Điều này hợp lý, đúng không? Thông thường, hộp sản phẩm càng lớn thì giá càng cao.

Nhân tiện, hãy thử hình dung ma trận tương quan của tất cả các cột bằng cách sử dụng gói `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Tuyệt vời hơn nhiều.

Một câu hỏi hay để đặt ra từ dữ liệu này sẽ là: '`Giá mà tôi có thể mong đợi cho một gói bí ngô nhất định là bao nhiêu?`' Hãy bắt đầu ngay thôi!

> Lưu ý: Khi bạn **`bake()`** công thức đã chuẩn bị **`pumpkins_prep`** với **`new_data = NULL`**, bạn sẽ trích xuất dữ liệu huấn luyện đã được xử lý (tức là đã được mã hóa). Nếu bạn có một tập dữ liệu khác, ví dụ như tập kiểm tra, và muốn xem cách công thức xử lý trước nó, bạn chỉ cần bake **`pumpkins_prep`** với **`new_data = test_set`**

## 4. Xây dựng mô hình hồi quy tuyến tính

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Đồ họa thông tin bởi Dasani Madipalli</figcaption>


<!--![Đồ họa thông tin bởi Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Bây giờ chúng ta đã xây dựng một công thức và thực sự xác nhận rằng dữ liệu sẽ được tiền xử lý một cách phù hợp, hãy cùng xây dựng một mô hình hồi quy để trả lời câu hỏi: `Giá của một gói bí ngô cụ thể sẽ là bao nhiêu?`

#### Huấn luyện mô hình hồi quy tuyến tính bằng tập huấn luyện

Như bạn có thể đã nhận ra, cột *price* là biến `kết quả` trong khi cột *package* là biến `dự đoán`.

Để thực hiện điều này, trước tiên chúng ta sẽ chia dữ liệu sao cho 80% vào tập huấn luyện và 20% vào tập kiểm tra, sau đó định nghĩa một công thức để mã hóa cột dự đoán thành một tập hợp các số nguyên, rồi xây dựng một đặc tả mô hình. Chúng ta sẽ không chuẩn bị và nướng công thức của mình vì đã biết rằng nó sẽ tiền xử lý dữ liệu như mong đợi.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Làm tốt lắm! Bây giờ chúng ta đã có một công thức và một mô tả mô hình, chúng ta cần tìm cách kết hợp chúng lại thành một đối tượng để trước tiên xử lý dữ liệu (prep+bake phía sau hậu trường), huấn luyện mô hình trên dữ liệu đã được xử lý và cũng cho phép các hoạt động xử lý hậu kỳ tiềm năng. Thật tuyệt vời để bạn yên tâm, đúng không!🤩

Trong Tidymodels, đối tượng tiện lợi này được gọi là [`workflow`](https://workflows.tidymodels.org/) và nó tiện lợi lưu giữ các thành phần mô hình của bạn! Đây là thứ mà chúng ta gọi là *pipelines* trong *Python*.

Vậy hãy gói gọn mọi thứ vào một workflow nhé!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 Thêm vào đó, một quy trình làm việc có thể được điều chỉnh/huấn luyện theo cách tương tự như một mô hình.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Từ đầu ra của mô hình, chúng ta có thể thấy các hệ số được học trong quá trình huấn luyện. Chúng đại diện cho các hệ số của đường hồi quy tốt nhất, giúp giảm thiểu tổng lỗi giữa biến thực tế và biến dự đoán.

#### Đánh giá hiệu suất mô hình bằng tập kiểm tra

Đã đến lúc xem mô hình hoạt động như thế nào 📏! Chúng ta làm điều này như thế nào?

Bây giờ chúng ta đã huấn luyện xong mô hình, có thể sử dụng nó để đưa ra dự đoán cho `test_set` bằng cách dùng `parsnip::predict()`. Sau đó, chúng ta có thể so sánh các dự đoán này với các giá trị nhãn thực tế để đánh giá mức độ hiệu quả (hoặc không hiệu quả!) của mô hình.

Hãy bắt đầu bằng cách tạo dự đoán cho tập kiểm tra, sau đó gắn các cột vào tập kiểm tra.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Vâng, bạn vừa huấn luyện một mô hình và sử dụng nó để đưa ra dự đoán! 🔮 Nó có tốt không? Hãy đánh giá hiệu suất của mô hình nhé!

Trong Tidymodels, chúng ta thực hiện việc này bằng cách sử dụng `yardstick::metrics()`! Đối với hồi quy tuyến tính, hãy tập trung vào các chỉ số sau:

-   `Root Mean Square Error (RMSE)`: Căn bậc hai của [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Đây là một chỉ số tuyệt đối có cùng đơn vị với nhãn (trong trường hợp này là giá của một quả bí ngô). Giá trị càng nhỏ, mô hình càng tốt (hiểu một cách đơn giản, nó đại diện cho mức giá trung bình mà dự đoán bị sai lệch!).

-   `Coefficient of Determination (thường được gọi là R-squared hoặc R2)`: Một chỉ số tương đối, trong đó giá trị càng cao, mô hình càng phù hợp. Về cơ bản, chỉ số này thể hiện mức độ mà mô hình có thể giải thích được sự biến thiên giữa giá trị dự đoán và giá trị thực tế của nhãn.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Hiệu suất của mô hình giảm đi. Hãy xem liệu chúng ta có thể có được một chỉ báo tốt hơn bằng cách trực quan hóa biểu đồ phân tán của gói hàng và giá, sau đó sử dụng các dự đoán đã thực hiện để vẽ một đường phù hợp nhất.

Điều này có nghĩa là chúng ta sẽ phải chuẩn bị và xử lý tập kiểm tra để mã hóa cột gói hàng, sau đó kết hợp nó với các dự đoán được tạo ra bởi mô hình của chúng ta.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Thật tuyệt! Như bạn có thể thấy, mô hình hồi quy tuyến tính không thực sự khái quát tốt mối quan hệ giữa một gói hàng và giá tương ứng của nó.

🎃 Chúc mừng, bạn vừa tạo ra một mô hình có thể giúp dự đoán giá của một vài loại bí ngô. Vườn bí ngô cho kỳ nghỉ của bạn sẽ thật đẹp. Nhưng có lẽ bạn có thể tạo ra một mô hình tốt hơn!

## 5. Xây dựng mô hình hồi quy đa thức

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Đồ họa thông tin bởi Dasani Madipalli</figcaption>


<!--![Đồ họa thông tin bởi Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Đôi khi dữ liệu của chúng ta không có mối quan hệ tuyến tính, nhưng chúng ta vẫn muốn dự đoán kết quả. Hồi quy đa thức có thể giúp chúng ta đưa ra dự đoán cho các mối quan hệ phi tuyến phức tạp hơn.

Hãy xem xét mối quan hệ giữa kích thước gói và giá cả trong bộ dữ liệu bí ngô của chúng ta. Mặc dù đôi khi có mối quan hệ tuyến tính giữa các biến - bí ngô có thể tích lớn hơn thì giá cao hơn - nhưng đôi khi những mối quan hệ này không thể được biểu diễn dưới dạng mặt phẳng hoặc đường thẳng.

> ✅ Đây là [một số ví dụ khác](https://online.stat.psu.edu/stat501/lesson/9/9.8) về dữ liệu có thể sử dụng hồi quy đa thức  
>  
> Hãy xem lại mối quan hệ giữa Loại bí ngô và Giá cả trong biểu đồ trước đó. Biểu đồ phân tán này có nhất thiết phải được phân tích bằng một đường thẳng không? Có lẽ là không. Trong trường hợp này, bạn có thể thử hồi quy đa thức.  
>  
> ✅ Đa thức là các biểu thức toán học có thể bao gồm một hoặc nhiều biến và hệ số  

#### Huấn luyện mô hình hồi quy đa thức bằng tập dữ liệu huấn luyện

Hồi quy đa thức tạo ra một *đường cong* để phù hợp hơn với dữ liệu phi tuyến.

Hãy xem liệu mô hình đa thức có hoạt động tốt hơn trong việc đưa ra dự đoán hay không. Chúng ta sẽ thực hiện quy trình tương tự như trước:

-   Tạo một công thức chỉ định các bước tiền xử lý cần thực hiện trên dữ liệu để chuẩn bị cho việc mô hình hóa, ví dụ: mã hóa các biến dự đoán và tính toán đa thức với bậc *n*

-   Xây dựng một mô hình cụ thể

-   Kết hợp công thức và mô hình cụ thể vào một quy trình làm việc

-   Tạo mô hình bằng cách khớp quy trình làm việc

-   Đánh giá mức độ hiệu quả của mô hình trên dữ liệu kiểm tra

Hãy bắt đầu ngay thôi!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Đánh giá hiệu suất mô hình

👏👏Bạn đã xây dựng một mô hình đa thức, hãy thực hiện dự đoán trên tập kiểm tra!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, hãy đánh giá cách mô hình hoạt động trên test_set bằng cách sử dụng `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Hiệu suất tốt hơn nhiều.

`rmse` đã giảm từ khoảng 7 xuống khoảng 3, cho thấy lỗi giữa giá thực tế và giá dự đoán đã giảm. Bạn có thể *hiểu một cách đơn giản* rằng trung bình, các dự đoán sai lệch khoảng \$3. `rsq` đã tăng từ khoảng 0.4 lên 0.8.

Tất cả các chỉ số này đều cho thấy rằng mô hình đa thức hoạt động tốt hơn nhiều so với mô hình tuyến tính. Làm tốt lắm!

Hãy xem liệu chúng ta có thể trực quan hóa điều này không!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Bạn có thể thấy một đường cong phù hợp với dữ liệu của mình tốt hơn! 🤩

Bạn có thể làm cho đường này mượt mà hơn bằng cách truyền một công thức đa thức vào `geom_smooth` như sau:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Giống như một đường cong mượt mà!🤩

Đây là cách bạn tạo một dự đoán mới:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Dự đoán của `polynomial model` thực sự hợp lý, dựa trên biểu đồ phân tán của `price` và `package`! Và nếu đây là một mô hình tốt hơn so với mô hình trước đó, khi nhìn vào cùng một dữ liệu, bạn cần dự trù ngân sách cho những quả bí ngô đắt tiền hơn này!

🏆 Làm tốt lắm! Bạn đã tạo hai mô hình hồi quy trong một bài học. Trong phần cuối cùng về hồi quy, bạn sẽ học về hồi quy logistic để xác định các danh mục.

## **🚀Thử thách**

Kiểm tra một số biến khác nhau trong notebook này để xem cách mối tương quan ảnh hưởng đến độ chính xác của mô hình.

## [**Câu hỏi sau bài giảng**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Ôn tập & Tự học**

Trong bài học này, chúng ta đã học về Hồi quy tuyến tính. Có những loại hồi quy quan trọng khác. Hãy tìm hiểu về các kỹ thuật Stepwise, Ridge, Lasso và Elasticnet. Một khóa học hay để học thêm là [Khóa học Stanford Statistical Learning](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Nếu bạn muốn tìm hiểu thêm về cách sử dụng framework Tidymodels tuyệt vời, hãy tham khảo các tài nguyên sau:

-   Trang web Tidymodels: [Bắt đầu với Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn và Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **CẢM ƠN ĐẾN:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) vì đã tạo ra những hình minh họa tuyệt vời giúp R trở nên thân thiện và hấp dẫn hơn. Tìm thêm các hình minh họa tại [bộ sưu tập của cô ấy](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, chúng tôi khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
